<a href="https://colab.research.google.com/github/francescoperagine/EyeInTheSkyNotebooks/blob/main/EyeInTheSky_YOLO_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install loguru==0.7.3 \
             tqdm==4.67.1 \
             typer==0.15.1 \
             ultralytics==8.3.78 \
             wandb==0.19.6

In [ ]:
import torch
from loguru import logger
import wandb
from google.colab import userdata, files
from ultralytics import YOLO, checks, settings
import time

In [ ]:
model_name = "yolo12n"
api_key = userdata.get("WANDB_API_KEY")

project_name = "EyeInTheSky"
experiment_name = "YOLO_VisDrone"
dataset_name = "VisDrone"

In [ ]:
try:
    device =  0 if torch.cuda.is_available() else "cpu"
except Exception as e:
    print(f"Error setting device: {e}")
print(f"Device: {device}")

In [ ]:
settings.update({"wandb": True})

In [ ]:
wandb.login(key=api_key)

In [ ]:
logger.info("Performing training for model...")
logger.info(checks())

In [ ]:
model = YOLO(f"{model_name}.pt")
results = model.train(data=f"{dataset_name}.yaml",
    workers=12,
    epochs=100,
    imgsz=640,
    device=device,
    patience=5,
    project=project_name,
    name=experiment_name,
    seed=42,
    plots=True,
    val=True
  )

In [ ]:
timestamp = time.strftime("%Y%m%d_%H%M%S")  # Get current time in YYYYMMDD_HHMMSS format
zip_filename = f"/content/{project_name}_{timestamp}.zip"

!zip -r "$zip_filename" /content/"$project_name"

In [ ]:
files.download(zip_filename)